# ESML: Scoring script: 
## Redeploy model  on AKS, adding predict_proba to scoring_script (based on AutoML generated)
- 1) `AutoMap datalake` & init ESML project
- 2) `Get earlier trained model`
- 3) `Get AutoML genereated scoring script`
- 4) `Customize scoring script & redeploy`




######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [2]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.inference_mode = False

unregister_all_datasets=False
if(unregister_all_datasets):
    p.unregister_all_datasets(p.ws) # For DEMO purpose

p.active_model = 10
p.describe()

In [3]:
p.describe()

Inference version: 1

 - ds01_titanic
projects/project002/01_titanic_model_clas/train/ds01_titanic/in/dev/2021/01/01/
projects/project002/01_titanic_model_clas/train/ds01_titanic/out/bronze/dev/
projects/project002/01_titanic_model_clas/train/ds01_titanic/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/01_titanic_model_clas/train/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_06_21/3f28069b18af4ca086e3fd48b83eda5b/
 

A)INFERENCE ONLINE: GOLD scored (example if realtime today)
projects/project002/01_titanic_model_clas/inference/1/scored/dev/2021_06_21/3f28069b18af4ca086e3fd48b83eda5b/
 

[B) USAGE]: to_score_folder_batch, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder)
B)INFERENCE BATCH: GOLD to score (example batch, datetime from config)
projects/pr

In [4]:
datastore = p.init() # 3) Automapping from datalake to Azure ML datasets

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = FALSE. [active_in_folder.json,active_scoring_in_folder.json] not found. 
 - Using [active_in_folder.json,active_scoring_in_folder.json] from ArgParse or GIT. No override from datalake settings

Inference mode (False = Training mode): False
Load data as Datasets....
ds01_titanic

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 1 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_titanic' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


train, evauluate, test = p.split_gold_3(0.6,label)

# 2) Azure ML `Edit AutoML generated scoring_script`
 - Path: `project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_dev.py`
 - DEMO custom script Path: `project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_custom.py`

In [5]:
#2a - Download from Azure
inference_config_to_override_and_inject, model, best_run = p.get_active_model_inference_config() # 1) You can override this scoring_script - get a baseline, then modify...

Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Loading AutoML config settings from: dev
Loading AutoML config settings from: dev
Package:azureml-automl-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-core, training version:1.30.0, current version:1.26.0
Package:azureml-dataprep, training version:2.15.1, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.13.0, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-defaults, training version:1.30.0, current version:1.26.0
Package:azureml-interpret, training version:1.30.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.30.0, current version:1.26.0
Package:azureml-telemetry, training ve

In [6]:
inference_config_to_override_and_inject.entry_script # Shows the scoring_script PATH, downloaded locally, ready to be edited and overridden

'c:\\Users\\jostrm\\OneDrive - Microsoft\\0_GIT\\2_My\\github\\azure-enterprise-scale-ml\\settings\\project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_dev.py'

In [7]:
import os
os.chdir(os.path.dirname(globals()['_dh'][0]))

#2b - Copy file, and edit, your CUSTOM scoring_script - see example "scoring_file_custom.py" ...and overwrite .entry_script
my_custom_script_instead = 'scoring_file_custom.py'
script_file_local = "./settings/project_specific/model/dev_test_prod/train/automl/"+my_custom_script_instead
script_file_abs = os.path.abspath(script_file_local)

inference_config_to_override_and_inject, model, best_run = p.get_active_model_inference_config(p.ws)
inference_config_to_override_and_inject.entry_script = script_file_abs
inference_config_to_override_and_inject.entry_script # Verify path

'c:\\Users\\jostrm\\OneDrive - Microsoft\\0_GIT\\2_My\\github\\azure-enterprise-scale-ml\\settings\\project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_custom.py'

# 3) ESML `Deploy model ONLINE` in `1 line of code` (AKS) 
- Deploy "offline" MODEL from old `run` in environment To →  `DEV`, `TEST` or `PROD` environment
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling`
    - Tip: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

In [8]:
# DEPLOY with custom InferenceConfig (custom scoring script)
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config_to_override_and_inject, True) #2) (model,inference_config, overwrite_endpoint=True,deployment_config=None):

Deploying model: AutoML8f39095707 with verison: 1 to environment: dev with overwrite_endpoint=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
image_build_compute = prj02-m01-dev
Found existing AksWebservice endpoint, deleting it, since overwrite=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster, esml-dev-prj02, using it.
Note: Autoscale_enabled=False, or since aks_dev_test=True in config, autoscaling is automatically shut off, e.g. overridden in config (since not supported) for environment dev
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-0

## 3b) ESML Test AKS webservice, `2 lines of code`


esml_dataset_bronze_df = p.DatasetByName("ds01_titanic").Bronze.to_pandas_dataframe() 
gold_train = p.save_gold(esml_dataset_bronze_df)  
label = "Survived"
train_set, validate_set, test_set = p.split_gold_3(0.6,label)
X_test, y_test, tags = p.get_gold_validate_Xy() 


In [9]:
label = p.active_model["label"]
to_score = None
try:
    X_test = p.GoldTest.to_pandas_dataframe()
    to_score = X_test.drop([label], axis=1)
    print(to_score.head()) # gold_test_1 = Dataset.get_by_name(ws, name=p.dataset_gold_test_name_azure)
except: 
    print ("you need to have splitted GOLD dataset, GoldTest need to exist. Change next cell from MARKDOWN, to CODE, and run that. Try this again... ")

   Pclass                                         Name     Sex   Age  \
0       3                              Mr. Sahid Nakid    male 20.00   
1       3                         Mr. Hanna Assi Barah    male 20.00   
2       3                            Mr. Edward Willey    male 21.00   
3       2                     Master. Viljo Hamalainen    male  0.67   
4       3  Mrs. Guillaume Joseph (Emma) de Messemaeker  female 36.00   

   Siblings/Spouses Aboard  Parents/Children Aboard  Fare  
0                        1                        1 15.74  
1                        0                        0  7.23  
2                        0                        0  7.55  
3                        1                        1 14.50  
4                        1                        0 17.40  


In [11]:
from baselayer_azure_ml import ComputeFactory
import json, pandas as pd
keyvault = p.ws.get_default_keyvault()
api_uri = keyvault.get_secret(name='esml-dev-p02-m01-api') # titanic = m01
api_key = keyvault.get_secret(name='esml-dev-p02-m01-apisecret') #

result_json = ComputeFactory.call_webservice_static(to_score, api_uri,api_key,firstRowOnly=False) # Simulate "REST call" (no ESML dependancy)
res_dict = json.loads(result_json.text) # json -> dictionary
df_res = pd.read_json(res_dict) # dictionary -> pandas
all_result = X_test.join(df_res) # features + result
all_result

Relying on you having the keys...


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,1,3,Mr. Sahid Nakid,male,20.00,1,1,15.74,0,0.02
1,1,3,Mr. Hanna Assi Barah,male,20.00,0,0,7.23,0,0.02
2,0,3,Mr. Edward Willey,male,21.00,0,0,7.55,0,0.22
3,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.50,1,0.92
4,1,3,Mrs. Guillaume Joseph (Emma) de Messemaeker,female,36.00,1,0,17.40,1,0.78
...,...,...,...,...,...,...,...,...,...,...
173,1,3,Mr. Albert Johan Moss,male,29.00,0,0,7.78,0,0.08
174,1,2,Mrs. (Elizabeth Anne Maidment) Mellinger,female,41.00,0,1,19.50,1,0.79
175,0,2,Mr. John William Gill,male,24.00,0,0,13.00,0,0.04
176,0,1,Mr. Mark Fortune,male,64.00,1,4,263.00,0,0.35


### Alt 2 - use compute factory, control to `get JSON back` instead of PANDAS. 
#### `No saving to LAKE`

In [12]:
result, model_version_used = p.compute_factory.call_webservice(to_score,False,False) # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")
df_res = pd.read_json(result)
all_result = to_score.join(df_res) # Need to join the FEATURES yourself, post webservice call (simulate no ESML dependancy in caller)
all_result

Note: Fetching keys automatically via workspace keyvault.


,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,3,Mr. Sahid Nakid,male,20.00,1,1,15.74,0,0.02
1,3,Mr. Hanna Assi Barah,male,20.00,0,0,7.23,0,0.02
2,3,Mr. Edward Willey,male,21.00,0,0,7.55,0,0.22
3,2,Master. Viljo Hamalainen,male,0.67,1,1,14.50,1,0.92
4,3,Mrs. Guillaume Joseph (Emma) de Messemaeker,female,36.00,1,0,17.40,1,0.78
...,...,...,...,...,...,...,...,...,...
173,3,Mr. Albert Johan Moss,male,29.00,0,0,7.78,0,0.08
174,2,Mrs. (Elizabeth Anne Maidment) Mellinger,female,41.00,0,1,19.50,1,0.79
175,2,Mr. John William Gill,male,24.00,0,0,13.00,0,0.04
176,1,Mr. Mark Fortune,male,64.00,1,4,263.00,0,0.35


### Alt 3 - ESML.call_webservice, `get PANDAS joined` dataframe
#### `Also saves to LAKE, automatically` 
- Requires p.init() or to manually set lake `p.lakestore = p.set_lake_as_datastore(p.ws)`

In [13]:
p.lakestore = p.set_lake_as_datastore(p.ws) # only needed if NOT p.init() is done
df = p.call_webservice(p.ws, to_score,"caller_id") # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")
df.head()

Using GEN2 as Datastore
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: Fetching keys automatically via workspace keyvault.
Saving scoring to lake for project folder project002 and inference_model_version: 1 ...
...

Saved DATA to score successfully in LAKE, as file 'to_score_caller_id.parquet'
Saved SCORED data in LAKE, as file 'scored_caller_id.parquet'


,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,3,Mr. Sahid Nakid,male,20.00,1,1,15.74,0,0.02
1,3,Mr. Hanna Assi Barah,male,20.00,0,0,7.23,0,0.02
2,3,Mr. Edward Willey,male,21.00,0,0,7.55,0,0.22
3,2,Master. Viljo Hamalainen,male,0.67,1,1,14.50,1,0.92
4,3,Mrs. Guillaume Joseph (Emma) de Messemaeker,female,36.00,1,0,17.40,1,0.78


## Optional: You can ALSO customize DeployConfig for AKS, programmatically...
- Can be done, `BUT recommendation is to use the "settings/" folder instead`, to override performance settings in .json per each evnironment (dev,test,prod)

In [14]:
deploy_config_to_override_and_inject = p.get_deploy_config_aks()  # 2) Optional: you can also override this - get a baseline, then modify...

print("description:", deploy_config_to_override_and_inject.description)
print("tags: ", deploy_config_to_override_and_inject.tags)
print("config: ", deploy_config_to_override_and_inject.print_deploy_configuration())
print("config - cpu_cores_limit (override what you want): ", deploy_config_to_override_and_inject.cpu_cores_limit)

Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: Autoscale_enabled=False, or since aks_dev_test=True in config, autoscaling is automatically shut off, e.g. overridden in config (since not supported) for environment dev
description: AKS webserice in DEV environment.
tags:  {'esml_model': '01_titanic_model_clas', 'esml_environment': 'dev'}
CPU requirement: 1, Memory requirement: 8GB, Number of replica: 1
config:  None
config - cpu_cores_limit (override what you want):  None


In [17]:
# DEPLOY with custom DeployConfig
#service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config_to_override_and_inject, True, own_deploy_config_to_inject) 
#inference_config, model, best_run = p.get_active_model_inference_config(ws) #  AutoML support 
#service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config,True) #2) (model,inference_config, overwrite_endpoint=True,deployment_config=None):

# END